# V1 取得 VE2 Agent 3.

In [12]:
from ckipnlp.pipeline import CkipPipeline, CkipDocument, CkipCorefPipeline
import pandas as pd
from tqdm import trange
import ast

## 載檔

In [13]:
file_full_name = "./v1_Total_Opinion.csv"

news_df = pd.read_csv(file_full_name)
all_output_file = "v2_Total_Opinion.csv"
opinion_after_cluster_file = "v1_cluster_opinions.csv"
topic_info_file = "v2_topic_info.csv"

In [14]:
news_df.head(1)

,id,article,category,company_id,reporter,timestamp,title,url_pattern,label,annotator,...,updated_at,lead_time,article_coref,people_name,ner,topic,representative_docs,one_to_one_pair,one_to_one_pair_count,opinions_list
0,883,"針對行政院長蘇貞昌、國民黨立委鄭麗文12日在立法院發生激烈爭執,兼任民進黨主席的總統蔡英文1...",政治,3,陶本和,1634110500,蔡英文發聲挺蘇貞昌 批鄭麗文情緒性發言對當事人不公,2100273,"[{""start"": 0, ""end"": 142, ""text"": ""針對行政院長蘇貞昌、國...",1,...,2022-02-03T15:02:18.598120Z,518.272,"針對行政院長蘇貞昌、國民黨立委鄭麗文12日在立法院發生激烈爭執,兼任民進黨主席的總統蔡英文1...","['蘇貞昌', '蔡英文', '民進黨', '行政院長', '中常會', '立法院', '國...","['蘇貞昌', '蔡英文', '台灣', '中國', '民進黨', '12日', '台灣',...",0,0,"[[{'start': 134, 'end': 138, 'text': '台灣人民', '...",9,"[[[{'start': 142, 'end': 145, 'text': '蔡英文', '..."


#### 輸入 ckip 帳號密碼

In [15]:
pipeline = CkipPipeline(opts={'con_parser': {'username': '', 'password': ''}})
coref_pipeline = CkipCorefPipeline(opts={'con_parser': {'username': '', 'password': ''}})

## 3. 產生 VE2 agent pair


### function 定義

In [16]:
def make_conparse(an_article, pipeline):
    doc = CkipDocument(raw=an_article)
    pipeline.get_conparse(doc)
    return doc.conparse[0]

In [17]:
def find_spice_index(an_article):
    return [i for i, char in enumerate(an_article) if char == ' ']

In [18]:
def make_agent_VE2_label(old_article, pipeline):

  spice_index_list = find_spice_index(old_article)
  conparse = make_conparse(old_article, pipeline)
  # print_tree_pic(old_article)

  article = ""
  start = 0
  end = 0
  # every sentence in article have 4 kinds VE2 and agent pairs
  one_to_one_pair = []
  # iterate the clause
  for ParseClause in conparse:
    sentence = ""
    if ParseClause.clause != '':
      VE2_count = 0
      agent_count = 0
      VE2_agent_clause_label = []
      VE2_candidate_label_list = []
      agent_candidate_label_list = []
      tree = ParseClause.to_tree()
      # iterate every node in sentence to find VE2 and agent
      for node in tree.all_nodes_itr():
        node_dict = node.to_dict()
        node_data = node_dict['data']

        node_id = node_dict['id']
        role = node_data['role']
        pos = node_data['pos']
        word = node_data['word']
        # change index when switch words
        if word != None:
          sentence += word
          start = end
          end = start + len(word)
        else:
          pass
        # find VE2
        if (('VE2' in pos) or ('VC' in pos) or ('VE11' in pos)):
          label = {'start': start, 'end': end, 'text': word, 'labels': ['VE2']}

          # VE2_agent_clause_label.append(label.copy())
          VE2_candidate_label_list.append(label.copy())
          VE2_count += 1
        # find agent
        if role != None:
          if 'agent' in role:
            if word == None:
              temp_word = ""
              sub_tree = tree.subtree(node_id)
              sub_node_list = sub_tree.all_nodes()
              sub_node_list.sort(key=lambda x: x.to_dict()['id'])
              for sub_node in sub_node_list:
                sub_node_dict = sub_node.to_dict()
                if sub_node_dict['data']['word'] != None:
                  temp_word += sub_node_dict['data']['word']
              temp_start = end
              temp_end = end + len(temp_word)
            else:
              temp_word = word
              temp_start = start
              temp_end = end
            label = {'start': temp_start, 'end': temp_end, 'text': temp_word, 'labels': ['agent']}
            # VE2_agent_clause_label.append(label.copy())
            agent_candidate_label_list.append(label.copy())
            agent_count += 1

    # change index when switch sentence
    sentence += ParseClause.delim
    # print(ParseClause.delim)
    start = end
    end = start + len(ParseClause.delim)
    article += sentence

    # check this sentence's VE2_agent pair belong to which kind
    if VE2_count >= 1 and agent_count >= 1:
      VE2_agent_clause_label.append(VE2_candidate_label_list[0])
      VE2_agent_clause_label.append(agent_candidate_label_list[0])
      one_to_one_pair.append(VE2_agent_clause_label)

  # change index when article have space
  for space_index in spice_index_list:
    article = article[:space_index] + " " + article[space_index:]
    for one_to_one_sentence_object in one_to_one_pair:
      for one_to_one_object in one_to_one_sentence_object:
        if one_to_one_object['start'] >= space_index:
          one_to_one_object['start'] += 1
          one_to_one_object['end'] += 1        
    
  return one_to_one_pair

## 6. 開始實作

In [19]:
"""Test for py_version"""
def top(file_full_name,pipeline):
    news_df = pd.read_csv(file_full_name)
    article_list = news_df['article_coref'].tolist()

    news_df['one_to_one_pair'] = ""
    news_df['one_to_one_pair_count'] = 0

    for i in trange(len(news_df)):
        if i == 1: # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            return
            # print(news_df)
            # return news_df
        try:
            news_df['one_to_one_pair'][i] = make_agent_VE2_label(article_list[i], pipeline)
        except:
            print('function return error in' + str(i))
            continue
        try:
            news_df['one_to_one_pair_count'][i] = len(news_df['one_to_one_pair'][i])
        except:
            print('one_to_one_pair_count error in' + str(i))
            continue
    return news_df

In [20]:
"""Test for py_version"""
def main():
    pipeline = CkipPipeline(opts={'con_parser': {'username': 'Howard_Wang', 'password': '*******'}})
    # get the file path from arg
    # parser = argparse.ArgumentParser()

    # parser.add_argument('--input_news_file_path',
    #                     #  default='./result/v2_Total_Opinion.csv',
    #                     help='enter input news data csv file path')

    # parser.add_argument('--output_all_output_file_path',
    #                     #  default='./result/v2_Total_Opinion.csv',
    #                     help='enter output_all_output_file_path')

    # args = parser.parse_args()

    # file_full_name = args.input_news_file_path # file_full_name = "./v1_Total_Opinion.csv"
    # all_output_file = args.output_all_output_file_path # all_output_file = "./v1_Total_Opinion.csv"
    # pathlib.Path(all_output_file).parent.mkdir(parents=True, exist_ok=True)
    top(file_full_name,pipeline)

In [21]:
"""Test for py_version"""
main()

  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
C:\Users\User\AppData\Local\Temp/ipykernel_16772/2005155955.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

### 3.

In [ ]:
article_list = news_df['article_coref'].tolist()

In [ ]:
news_df['one_to_one_pair'] = ""
news_df['one_to_one_pair_count'] = 0

for i in trange(len(news_df)):
    if i == 3:
        break
    try:
        news_df['one_to_one_pair'][i] = make_agent_VE2_label(article_list[i], pipeline)
    except:
        print('function return error in' + str(i))
        continue
    try:
        news_df['one_to_one_pair_count'][i] = len(news_df['one_to_one_pair'][i])
    except:
        print('one_to_one_pair_count error in' + str(i))
        continue
    

  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
C:\Users\User\AppData\Local\Temp/ipykernel_12744/2872328293.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [ ]:
news_df.head(3)

,id,article,category,company_id,reporter,timestamp,title,url_pattern,label,annotator,...,updated_at,lead_time,article_coref,people_name,ner,topic,representative_docs,one_to_one_pair,one_to_one_pair_count,opinions_list
0,883,"針對行政院長蘇貞昌、國民黨立委鄭麗文12日在立法院發生激烈爭執,兼任民進黨主席的總統蔡英文1...",政治,3,陶本和,1634110500,蔡英文發聲挺蘇貞昌 批鄭麗文情緒性發言對當事人不公,2100273,"[{""start"": 0, ""end"": 142, ""text"": ""針對行政院長蘇貞昌、國...",1,...,2022-02-03T15:02:18.598120Z,518.272,"針對行政院長蘇貞昌、國民黨立委鄭麗文12日在立法院發生激烈爭執,兼任民進黨主席的總統蔡英文1...","['蘇貞昌', '蔡英文', '民進黨', '行政院長', '中常會', '立法院', '國...","['蘇貞昌', '蔡英文', '台灣', '中國', '民進黨', '12日', '台灣',...",0,0,"[[{'start': 53, 'end': 55, 'text': '力挺', 'labe...",14,"[[[{'start': 142, 'end': 145, 'text': '蔡英文', '..."
1,862,"行政院長蘇貞昌12日在立法院總質詢時,怒斥國民黨立委鄭麗文「袂見笑」、「當時還要拜託我去幫你...",政治,3,呂晏慈,1634092560,蘇貞昌酸鄭麗文「民進黨叛徒」 他反譏:蔡英文也當國民黨閣員,2100056,"[{""start"": 0, ""end"": 51, ""text"": ""行政院長蘇貞昌12日在立...",1,...,2022-02-03T12:47:43.626597Z,30432.838,"行政院長蘇貞昌12日在立法院總質詢時,怒斥國民黨立委鄭麗文「袂見笑」、「當時還要拜託我去幫你...","['蘇貞昌', '行政院長', '鄭麗文', '國民黨', '蔡英文', '李德維', '立...","['蘇貞昌', '行政院長', '蘇', '鄭麗文', '國民黨', '蔡英文', '25、...",3,0,"[[{'start': 61, 'end': 63, 'text': '舉行', 'labe...",16,"[[[{'start': 86, 'end': 94, 'text': '費鴻泰', 'la..."
2,854,"台灣基進立委陳柏惟遭罷免團體掛看板質疑「反對採購疫苗」,立法院副院長蔡其昌4日反駁,陳柏惟反...",政治,3,呂晏慈,1634090220,蔡其昌質疑「想買中國科興疫苗」 國民黨團今按鈴控告妨害名譽,2100001,"[{""start"": 0, ""end"": 27, ""text"": ""台灣基進立委陳柏惟遭罷免...",1,...,2022-02-03T06:27:27.039925Z,7616.256,"台灣基進立委陳柏惟遭罷免團體掛看板質疑「反對採購疫苗」,立法院副院長蔡其昌4日反駁,陳柏惟反...","['蔡其昌', '國民黨團', '中國科興', '立法院', '莫德納', '陳柏惟', '...","['蔡其昌', '國民黨團', '科興', '中國科興', '今天', '蔡其昌', '嬌生...",3,0,"[[{'start': 10, 'end': 12, 'text': '罷免', 'labe...",6,"[[[{'start': 0, 'end': 9, 'text': '陳柏惟', 'labe..."


In [ ]:
news_df.to_csv('./v2_Total_Opinion.csv', index=False)